In [0]:
from google.colab import drive
drive.mount('/content/drive')
import re
import numpy as np
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read file truyen_Kieu.txt and clean text.

In [0]:
path = '/content/drive/My Drive/dataset/truyen_Kieu.txt'
import re

with open(path) as file_text:
  i = 0
  text = file_text.read()
  text = re.sub('[0-9]\.? ?\.? ?|[\.,?!]','', text)

In [0]:
print('Length of text : {} characters'.format(len(text)))

Length of text : 101360 characters


In [0]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

125 unique characters


# Create vocabulary (include space and newline character)

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  "'" :   2,
  '-' :   3,
  ':' :   4,
  ';' :   5,
  'A' :   6,
  'B' :   7,
  'C' :   8,
  'D' :   9,
  'E' :  10,
  'G' :  11,
  'H' :  12,
  'K' :  13,
  'L' :  14,
  'M' :  15,
  'N' :  16,
  'O' :  17,
  'P' :  18,
  'Q' :  19,
  ...
}


In [0]:
# Show how the first 14 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:14]), text_as_int[:14]))

'Trăm năm trong' ---- characters mapped to int ---- > [22 43 70 38  1 39 70 38  1 45 43 40 39 33]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [0]:
sequences = char_dataset.batch(seq_length, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Trăm năm trong cõi người ta\nChữ tài chữ mệnh khéo là ghét nhau\nTrải qua một cuộc bể dâu\nNhững điều t'
'rông thấy mà đau đớn lòng\nLạ gì bỉ sắc tư phong\nTrời xanh quen thói má hồng đánh ghen\nCảo thơm lần g'
'iở trước đèn\nPhong tình có lục còn truyền sử xanh\nRằng năm Gia Tĩnh triều Minh\nBốn phương phẳng lặng'
' hai kinh vững vàng\nCó nhà viên ngoại họ Vương\nGia tư nghĩ cũng thường thường bực trung\nMột trai con'
' thứ rốt lòng\nVương Quan là chữ nối dòng nho gia\nĐầu lòng hai ả tố nga\nThúy Kiều là chị em là Thúy V'


# Create input sequence and target sequence.

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Trăm năm trong cõi người ta\nChữ tài chữ mệnh khéo là ghét nhau\nTrải qua một cuộc bể dâu\nNhững điều '
Target data: 'răm năm trong cõi người ta\nChữ tài chữ mệnh khéo là ghét nhau\nTrải qua một cuộc bể dâu\nNhững điều t'


# Create a tf.data dataset

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# Create simple model

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (64, None, 256)           32000     
_________________________________________________________________
gru_11 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_11 (Dense)             (64, None, 125)           128125    
Total params: 4,098,429
Trainable params: 4,098,429
Non-trainable params: 0
_________________________________________________________________


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 125)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8289127


In [0]:
model.compile(optimizer='adam', loss=loss)

# Checkpoints

In [0]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=150

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)
model1 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model1.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model1.build(tf.TensorShape([1, None]))
model1.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (1, None, 256)            32000     
_________________________________________________________________
gru_14 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
dense_14 (Dense)             (1, None, 125)            128125    
Total params: 4,098,429
Trainable params: 4,098,429
Non-trainable params: 0
_________________________________________________________________


# Eveluate Model

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
 
  for i in range(num_generate):
      predictions = model(input_eval)

      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)
      
      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model1, start_string=u"Trăm"))

Trămới biết nghĩ sau
Người lên ngựa kẻ chia bào
Rừng phong thuận dùng dãy đòng bỏ chững nhiều
Đặm trường thần sách mồn mang
Trong nào nỡ dứt nghĩa người ra đi
ông rằng: Bỉ thử nhất thì
Tu hàng mở cỏ nhìn lòng
Tạ ân lạy trước Từ công:
Chút thân bồ liễu nành hoa chưa về
Cửa ngoài vội rủ rèm the
Xăm xăm băng lối nước cao thâm dường
Lại tìm những chốn đoạn trường mà đi
Hết nạn ấy đến nạn kiếp phương trời đi tình biết một mình mình hay
Làm cho sốn đọc thoang thịức đường tơ lụ mười viên tai
Điều đâu nói lạ dường này
Sự nàng đã biết tình chi đây\
Rằng: Lòng đương thổn thức đầy
Tơ duyên còn vướng tơi bời
Mặt duyên đâu Hãy chơ người thước cao
Được lời mụ mới tùy cơ nguyền thầm xuân đã quên thân
Một nhà để chị riêng oan một mình
Chẳng ngờ gã Mã Giám Sinh
Với chi lá chống những người 
Thư khói việc nàng về xem ý tứ nhà
Sự mình cũng rắp lân la giải giở nhìn đâu chân gia
Thanh lời nàng mới lựa dây
Nỉ nha đã dạt xa xăn bót tỏi trời tri 
đảo nhiều lại gặp thế nào
Bại ra thế ấy vịnh vào thế kia 
Phong